In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

### Aggregate to unique zip, remove zips without all 6 years, shift outcome, and split into train and test.

Aggregate to unique zip

In [2]:
df = pd.read_csv('../data/merged_med_seasonal_2011_2016_v2.csv').drop(columns=['Unnamed: 0'])

zip_df_pre = df.drop(columns=['year','ZIP'])

# don't need this anymore - there is monthly population in  med_monthly
# med_pop = pd.read_csv('medicare_pop_aggregated_2011_2016.csv')

med_monthly = pd.read_csv('../data/medicare_deaths_monthly_2011_2016.csv').drop(columns=['Unnamed: 0'])

In [3]:
def apply_month(m):
    month_dict = {'JAN':1,'FEB':2,'MAR':3,'APR':4,
             'MAY':5,'JUN':6,'JUL':7,'AUG':8,
              'SEP':9,'OCT':10,'NOV':11,'DEC':12}
    val = month_dict[m]
    return val

zip_df_pre['AMONTH'] = zip_df_pre['AMONTH'].apply(apply_month)

In [4]:
merged_df2 = zip_df_pre.merge(med_monthly, how='inner', left_on=['zip','AYEAR','AMONTH'], right_on=['zip','year','month'])

Shift zip (removing those without all 6 years

In [5]:
def remove_zip_and_shift(df):
    # find zips with missing years and months between 2011 - 2016
    zipp = list(np.unique(df.zip))
    zs = []
    for z in zipp:
        sub = df[df.zip == z]
        # make sure each zip has all years
        if np.unique(sub.AYEAR).shape[0] != 6:
            zs.append(z)
        
        # make sure each zip has all months within each year
        for year in np.unique(sub.AYEAR):
            sub2 = sub[sub.AYEAR == year]
            if np.unique(sub2.AMONTH).shape[0] != 12:
                zs.append(z)
    
    # find zips that have all 6 years
    zip_to_keep = list(np.setdiff1d(list(df.zip.values),zs))
    
    # only keep data with zips for all 6 years and all 12 months
    sub_zip = df[df.zip.isin(zip_to_keep)]
    
    # shift yearly death count and dead variables by 1 year, so outcome is adjusted
    # also shifts monthly death count variable by 1 year
    zip_df_new = sub_zip.copy()
    
    # sort
    zip_df_new_sorted = zip_df_new.groupby(zip_df_new['zip']).apply(lambda x: x.sort_values(by=['AYEAR','AMONTH'],ascending=True))
    
    # shift
    zip_df_new_sorted['death_rate_next_year'] = zip_df_new_sorted.groupby(zip_df_new_sorted['zip'])['dead'].shift(-12) # yearly
    zip_df_new_sorted['deaths_next_year'] = zip_df_new_sorted.groupby(zip_df_new_sorted['zip'])['death'].shift(-12) # yearly
    zip_df_new_sorted['deaths_next_month'] = zip_df_new_sorted.groupby(zip_df_new_sorted['zip'])['deaths'].shift(-1) # monthly
    return zip_df_new_sorted

In [6]:
# z = merged_df2[merged_df2.popdensity==0].zip.values
# merged_df3 =  merged_df2.loc[~merged_df2.zip.isin(z)]

merged_df2['CVD_pdx2dx_25'] = merged_df2['CVD_pdx2dx_25'] +merged_df2['CHF_pdx2dx_25'] +merged_df2['AMI_pdx2dx_25'] + merged_df2['CSD_pdx2dx_25'] 
merged_df2['Stroke_pdx2dx_25'] = merged_df2['Stroke_pdx2dx_25'] + merged_df2['Hemo_Stroke_pdx2dx_25'] +merged_df2['Ischemic_stroke_pdx2dx_25']

neos = ['neo_140_149', 'neo_150_159', 'neo_160_165', 'neo_170_176', 'neo_179_189',
       'neo_190_199', 'neo_200_209', 'neo_230_234',
       'neo_235_238', 'neo_239']

merged_df2['malignant_neoplasms'] = 0
for n in neos:
    merged_df2['malignant_neoplasms']+= merged_df2[n] 


In [7]:
# zs = np.unique(merged_df2[merged_df2.popdensity==0].zip)
# merged_df3 = merged_df2[~merged_df2.zip.isin(zs)]
# merged_df3.shape, merged_df2.shape

In [8]:
# merged_df4 = merged_df3.copy()

# cols_to_adj = ['death', 
#                'ICU_DAY', 'CCI_DAY', 'LOS','Parkinson_pdx2dx_25', 'Alzheimer_pdx2dx_25', 
#                'Dementia_pdx2dx_25','CHF_pdx2dx_25', 'AMI_pdx2dx_25', 'COPD_pdx2dx_25', 
#                'DM_pdx2dx_25', 'Stroke_pdx2dx_25', 'CVD_pdx2dx_25', 'CSD_pdx2dx_25',
#                'Ischemic_stroke_pdx2dx_25', 'Hemo_Stroke_pdx2dx_25', 'neo_140_149',
#                'neo_150_159', 'neo_160_165', 'neo_170_176', 'neo_179_189',
#                'neo_190_199', 'neo_200_209', 'neo_210_229', 'neo_230_234',
#                'neo_235_238', 'neo_239', 'malignant_neoplasms',
#                'm_count', 'f_count','white_count', 'black_count',
#                'hispanic_count', 'asian_count', 'native_count','deaths']


# for c in cols_to_adj:
#     merged_df4[c] = merged_df4[c]/merged_df4.population
# zip_df2 = remove_zip_and_shift(merged_df4).drop(columns=['zip']).reset_index().drop(columns=['level_1'])

In [7]:
zip_df2 = remove_zip_and_shift(merged_df2).drop(columns=['zip']).reset_index().drop(columns=['level_1'])

Monthly

In [10]:
# cols_to_adj = ['f_count', 'm_count', 'white_count','black_count', 
#         'hispanic_count', 'asian_count','native_count']

# new_cols = ['f_prop', 'm_prop', 'white_prop','black_prop', 
#         'hispanic_prop', 'asian_prop','native_prop']

# for i in range(len(cols_to_adj)):
#     zip_df3[new_cols[i]] = zip_df3[cols_to_adj[i]]/zip_df3['population']

In [11]:
zip_test_monthly = zip_df2[zip_df2.AYEAR == 2016]
zip_train_monthly = zip_df2[zip_df2.AYEAR != 2016]

In [12]:
zip_train_monthly.to_csv('zip_train_monthly.csv') # rolling averages, aggregated by zip, month, year, outcome shifted
zip_test_monthly.to_csv('zip_test_monthly.csv')

# zip_train_monthly.to_csv('zip_train_monthly_v2.csv') # rolling averages, aggregated by zip, month, year, outcome shifted, adjusted for population
# zip_test_monthly.to_csv('zip_test_monthly_v2.csv')

Yearly

In [17]:
# be careful for which ones change monthly - only sum those that change monthly, not yearly or else its double counted
col_to_agg = {'sex':'mean', 'age':'mean', 'statecode':max,
       'dual':max, 'death':max, 'dead':'mean', 'poverty': 'mean', 'popdensity': 'mean', 
       'medianhousevalue': 'mean','pct_blk': 'mean', 'medhouseholdincome' : 'mean', 'pct_owner_occ': 'mean',
       'hispanic': 'mean','education': 'mean', 'smoke_rate': 'mean', 'mean_bmi': 'mean', 
       'rmax': 'mean', 'pr': 'mean', 
       'race_0':'mean', 'race_1':'mean', 'race_2':'mean', 'race_3':'mean', 
       'race_4':'mean', 'race_5':'mean', 'race_6':'mean',
       'ICU_DAY':sum, 'CCI_DAY':sum, 'LOS':'mean', 'Parkinson_pdx2dx_25':sum,
       'Alzheimer_pdx2dx_25':sum, 'Dementia_pdx2dx_25':sum, 'CHF_pdx2dx_25':sum,
       'AMI_pdx2dx_25':sum, 'COPD_pdx2dx_25':sum, 'DM_pdx2dx_25':sum, 'Stroke_pdx2dx_25':sum,
       'CVD_pdx2dx_25':sum, 'CSD_pdx2dx_25':sum, 'Ischemic_stroke_pdx2dx_25':sum,
       'Hemo_Stroke_pdx2dx_25':sum, 'neo_140_149':sum, 'neo_150_159':sum, 'neo_160_165':sum,
       'neo_170_176':sum, 'neo_179_189':sum, 'neo_190_199':sum, 'neo_200_209':sum,
       'neo_210_229':sum, 'neo_230_234':sum, 'neo_235_238':sum, 'neo_239':sum, 
       'pm25_summer_4y_avg':'mean', 'pm25_winter_4y_avg':'mean', 'pm25_fall_4y_avg':'mean', 'pm25_spring_4y_avg': 'mean',
       'ozone_summer_4y_avg':'mean', 'ozone_winter_4y_avg':'mean', 'ozone_fall_4y_avg': 'mean', 'ozone_spring_4y_avg': 'mean',
       'no2_summer_4y_avg':'mean', 'no2_winter_4y_avg': 'mean', 'no2_fall_4y_avg': 'mean', 'no2_spring_4y_avg': 'mean', 
       'summer_tmmx_4y_avg': 'mean','summer_rmax_4y_avg': 'mean', 'winter_tmmx_4y_avg': 'mean', 'winter_rmax_4y_avg': 'mean',
        'm_count':max, 'f_count':max, 'mean_age':'mean',
        'population':max, # population is monthly
       'white_count':'mean', 'black_count':'mean', 'hispanic_count':'mean', 'asian_count':'mean',
       'native_count':'mean','deaths_next_year':max}

col_to_agg2 = {'sex':'mean', 'age':'mean', 'statecode':max,
       'dual':max, 'death':max, 'dead':'mean', 'poverty': 'mean', 'popdensity': 'mean', 
       'medianhousevalue': 'mean','pct_blk': 'mean', 'medhouseholdincome' : 'mean', 'pct_owner_occ': 'mean',
       'hispanic': 'mean','education': 'mean', 'smoke_rate': 'mean', 'mean_bmi': 'mean', 
       'rmax': 'mean', 'pr': 'mean', 
       'race_0':'mean', 'race_1':'mean', 'race_2':'mean', 'race_3':'mean', 
       'race_4':'mean', 'race_5':'mean', 'race_6':'mean',
       'ICU_DAY':sum, 'CCI_DAY':sum, 'LOS':'mean', 'Parkinson_pdx2dx_25':sum,
       'Alzheimer_pdx2dx_25':sum, 'Dementia_pdx2dx_25':sum, 'CHF_pdx2dx_25':sum,
       'AMI_pdx2dx_25':sum, 'COPD_pdx2dx_25':sum, 'DM_pdx2dx_25':sum, 'Stroke_pdx2dx_25':sum,
       'CVD_pdx2dx_25':sum, 'CSD_pdx2dx_25':sum, 'Ischemic_stroke_pdx2dx_25':sum,
       'Hemo_Stroke_pdx2dx_25':sum, 'neo_140_149':sum, 'neo_150_159':sum, 'neo_160_165':sum,
       'neo_170_176':sum, 'neo_179_189':sum, 'neo_190_199':sum, 'neo_200_209':sum,
       'neo_210_229':sum, 'neo_230_234':sum, 'neo_235_238':sum, 'neo_239':sum, 
        'malignant_neoplasms':sum, 
       'pm25_summer_4y_avg':'mean', 'pm25_winter_4y_avg':'mean', 'pm25_fall_4y_avg':'mean', 'pm25_spring_4y_avg': 'mean',
       'ozone_summer_4y_avg':'mean', 'ozone_winter_4y_avg':'mean', 'ozone_fall_4y_avg': 'mean', 'ozone_spring_4y_avg': 'mean',
       'no2_summer_4y_avg':'mean', 'no2_winter_4y_avg': 'mean', 'no2_fall_4y_avg': 'mean', 'no2_spring_4y_avg': 'mean', 
       'summer_tmmx_4y_avg': 'mean','summer_rmax_4y_avg': 'mean', 'winter_tmmx_4y_avg': 'mean', 'winter_rmax_4y_avg': 'mean',
        'm_count':'mean', 'f_count':'mean', 'mean_age':'mean',
        'population':max, # population is monthly
       'white_count':'mean', 'black_count':'mean', 'hispanic_count':'mean', 'asian_count':'mean',
       'native_count':'mean','deaths_next_year':max, 'death_rate_next_year':'mean'}

zip_df_yearly = zip_df2.groupby(['zip', 'AYEAR']).aggregate(col_to_agg2)
zip_df_yearly = zip_df_yearly.reset_index()

In [18]:
zip_train_yearly  = zip_df_yearly[zip_df_yearly.AYEAR != 2016]
zip_test_yearly = zip_df_yearly[zip_df_yearly.AYEAR == 2016]

zip_train_yearly.to_csv('zip_train_yearly.csv') # rolling averages, aggregated by zip, month, year, outcome shifted for rnn
zip_test_yearly.to_csv('zip_test_yearly.csv')

# zip_train_yearly.to_csv('zip_train_yearly_v2.csv') # rolling averages, aggregated by zip, month, year, outcome shifted for rnn
# zip_test_yearly.to_csv('zip_test_yearly_v2.csv')

In [36]:
# monthly visualizations
merged_df3.to_csv('merged_df3_monthly.csv')

In [31]:
# create data for visualization
col_to_agg3 = {'sex':'mean', 'age':'mean', 'statecode':max,
       'dual':max, 'death':max, 'dead':'mean', 'poverty': 'mean', 'popdensity': 'mean', 
       'medianhousevalue': 'mean','pct_blk': 'mean', 'medhouseholdincome' : 'mean', 'pct_owner_occ': 'mean',
       'hispanic': 'mean','education': 'mean', 'smoke_rate': 'mean', 'mean_bmi': 'mean', 
       'rmax': 'mean', 'pr': 'mean', 
       'race_0':'mean', 'race_1':'mean', 'race_2':'mean', 'race_3':'mean', 
       'race_4':'mean', 'race_5':'mean', 'race_6':'mean',
       'ICU_DAY':sum, 'CCI_DAY':sum, 'LOS':'mean', 'Parkinson_pdx2dx_25':sum,
       'Alzheimer_pdx2dx_25':sum, 'Dementia_pdx2dx_25':sum, 'CHF_pdx2dx_25':sum,
       'AMI_pdx2dx_25':sum, 'COPD_pdx2dx_25':sum, 'DM_pdx2dx_25':sum, 'Stroke_pdx2dx_25':sum,
       'CVD_pdx2dx_25':sum, 'CSD_pdx2dx_25':sum, 'Ischemic_stroke_pdx2dx_25':sum,
       'Hemo_Stroke_pdx2dx_25':sum, 'neo_140_149':sum, 'neo_150_159':sum, 'neo_160_165':sum,
       'neo_170_176':sum, 'neo_179_189':sum, 'neo_190_199':sum, 'neo_200_209':sum,
       'neo_210_229':sum, 'neo_230_234':sum, 'neo_235_238':sum, 'neo_239':sum, 
        'malignant_neoplasms':sum, 
       'pm25_summer_4y_avg':'mean', 'pm25_winter_4y_avg':'mean', 'pm25_fall_4y_avg':'mean', 'pm25_spring_4y_avg': 'mean',
       'ozone_summer_4y_avg':'mean', 'ozone_winter_4y_avg':'mean', 'ozone_fall_4y_avg': 'mean', 'ozone_spring_4y_avg': 'mean',
       'no2_summer_4y_avg':'mean', 'no2_winter_4y_avg': 'mean', 'no2_fall_4y_avg': 'mean', 'no2_spring_4y_avg': 'mean', 
       'summer_tmmx_4y_avg': 'mean','summer_rmax_4y_avg': 'mean', 'winter_tmmx_4y_avg': 'mean', 'winter_rmax_4y_avg': 'mean',
        'm_count':'mean', 'f_count':'mean', 'mean_age':'mean',
        'population':'mean', # population is monthly
       'white_count':'mean', 'black_count':'mean', 'hispanic_count':'mean', 'asian_count':'mean',
       'native_count':'mean','deaths':sum}

merged_df3_yearly = merged_df3.groupby(['zip', 'AYEAR']).aggregate(col_to_agg3)
merged_df3_yearly = merged_df3_yearly.reset_index()

In [34]:
merged_df3_yearly.to_csv('merged_df3.csv')

## Create Separate Datasets for lower and higher population

In [28]:
zip_df_yearly_lower_pop = zip_df_yearly[zip_df_yearly.population < 12295]
zip_df_yearly_higher_pop = zip_df_yearly[zip_df_yearly.population >= 12295]

In [30]:
zip_train_yearly_lower_pop  = zip_df_yearly_lower_pop[zip_df_yearly_lower_pop.AYEAR != 2016]
zip_test_yearly_lower_pop = zip_df_yearly_lower_pop[zip_df_yearly_lower_pop.AYEAR == 2016]

zip_train_yearly_lower_pop.to_csv('zip_train_yearly_lower_pop.csv') # rolling averages, aggregated by zip, month, year, outcome shifted for rnn
zip_test_yearly_lower_pop.to_csv('zip_test_yearly_lower_pop.csv')

zip_train_yearly_higher_pop  = zip_df_yearly_higher_pop[zip_df_yearly_higher_pop.AYEAR != 2016]
zip_test_yearly_higher_pop = zip_df_yearly_higher_pop[zip_df_yearly_higher_pop.AYEAR == 2016]

zip_train_yearly_higher_pop.to_csv('zip_train_yearly_higher_pop.csv') # rolling averages, aggregated by zip, month, year, outcome shifted for rnn
zip_test_yearly_higher_pop.to_csv('zip_test_yearly_higher_pop.csv')

In [9]:
# sort merged-df2
# merged_df2_sorted = merged_df2.groupby(merged_df2['zip']).apply(lambda x: x.sort_values(by=['AYEAR','AMONTH'],ascending=True))

# find change in deaths
# zip_df2['change_in_monthly_deaths'] = zip_df2['deaths'] - merged_df2_sorted['deaths']

Split into train and test: i.e. 2011-2015, and 2016

Save Results

In [18]:
# zip_train.to_csv('zip_train.csv') - no rolling averages
# zip_test.to_csv('zip_test.csv')
# zip_train.to_csv('zip_train_v2.csv') -rolling averages, aggregated by zip, year
# zip_test.to_csv('zip_test_v2.csv')
# zip_train.to_csv('zip_train_v3.csv') # rolling averages, aggregated by zip, month, year, outcomem not shifted for rnn
# zip_test.to_csv('zip_test_v3.csv')
zip_train.to_csv('zip_train_v4.csv') # rolling averages, aggregated by zip, month, year, outcome shifted for rnn
zip_test.to_csv('zip_test_v4.csv')